Program ini digunakan untuk meningkatkan performa dari https://github.com/andikarachman/RNN-Twitter-Sentiment-Analysis/blob/master/RNN_Twitter_Sentiment_Analysis.ipynb


Akurasi awal adalah 89% menjadi 95% dengan menambahkan stemming dan beberapa data processing tambahan. Hal ini membuktikan bahwa, data yang diproses dengan baik, akan menghasilkan hasil yang baik juga

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('Tweets.csv')

In [ ]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [ ]:
review = df['text']
review.head()

0                  @VirginAmerica What @dhepburn said.
1    @VirginAmerica plus you've added commercials t...
2    @VirginAmerica I didn't today... Must mean I n...
3    @VirginAmerica it's really aggressive to blast...
4    @VirginAmerica and it's a really big bad thing...
Name: text, dtype: object

In [ ]:
label = df['airline_sentiment']
label.head()

0     neutral
1    positive
2     neutral
3    negative
4    negative
Name: airline_sentiment, dtype: object

In [ ]:
import re

In [ ]:
review[0]

'@VirginAmerica What @dhepburn said.'

In [ ]:
def removal(word):
  word =  word.strip('\'"?!,.():;-') #menghilangkan tanda baca
  word = re.sub(r'(.)\1+', r'\1\1', word) #menghilangkan apabila ada duplikat huruf lebih dari 2
  #word = re.sub(r'(-|\')', '', word) 
  return word

In [ ]:
word = '--funn--nn''nnnny!!!!-!'
removal(word)

'funn--nny'

In [ ]:
def emoji(word):
  word = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))',' POS_EMO ',word)
  word = re.sub(r'(:\s?D|:-D|x-?D|X-?D)',' POS_EMO ',word)
  word = re.sub(r'(<3|:\*)',' POS_EMO ',word)
  word = re.sub(r'(<3|:\*)',' POS_EMO ',word)
  word = re.sub(r'(;-?\)|;-?D|\(-?;)',' POS_EMO ',word)
  word = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)',' NEG_EMO ',word)
  word = re.sub(r'(:,\(|:\'\(|:"\()',' NEG_EMO ',word)
  return word

In [ ]:
word = ':) :( <3 :D'
emoji(word)

' POS_EMO   NEG_EMO   POS_EMO   POS_EMO '

In [ ]:
def is_valid_word(word):
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

In [ ]:
is_valid_word('a@aaaa@@@')

False

In [ ]:
is_valid_word('akua')

True

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
wo = 'finding'
wo = str(ps.stem(wo))
wo

'find'

In [ ]:
def prep(word):
  word = re.sub(r'((www\.[\S]+)|(https?://[\S]+))',' URL ',word)
  word = re.sub(r'@[\S]+','MENTION_USER',word)
  word = re.sub(r'#(\S+)', r' \1 ',word)
  word = re.sub(r'\brt\b', '', word)
  word = re.sub(r'\.{2,}',' ',word)
  word = word.strip(' "\'')
  word = emoji(word)
  word = re.sub(r'\s+',' ',word)

  proc_word = []
  words = word.split()

  for w in words:
    w = removal(w)
    if is_valid_word(w):
      w = str(ps.stem(w))
      proc_word.append(w)
  return ' '.join(proc_word)

In [ ]:
prep("@yafiarkan https://yafiarkan.github.io  looking for #datascience job:)")

'mention_us url look for datasci job pos_emo'

In [ ]:
prep('im using this to make a checking trial')

'im use thi to make a check trial'

In [ ]:
all_text = np.array(review)
all_text[0:20]

array(['@VirginAmerica What @dhepburn said.',
       "@VirginAmerica plus you've added commercials to the experience... tacky.",
       "@VirginAmerica I didn't today... Must mean I need to take another trip!",
       '@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse',
       "@VirginAmerica and it's a really big bad thing about it",
       "@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.\nit's really the only bad thing about flying VA",
       '@VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)',
       '@VirginAmerica Really missed a prime opportunity for Men Without Hats parody, there. https://t.co/mWpG7grEZP',
       "@virginamerica Well, I didn't…but NOW I DO! :-D",
       "@VirginAmerica it was amazing, and arrived an hour early. You're too good to me.",
       '@VirginAmerica did you know that suicide is the second leading cause of

In [ ]:
len(all_text)

14640

In [ ]:
for i in range(len(all_text)):
  all_text[i] = prep(all_text[i])
all_text[0:20]

array(['mention_us what mention_us said',
       'mention_us plu ad commerci to the experi tacki',
       'mention_us I today must mean I need to take anoth trip',
       'mention_us realli aggress to blast obnoxi entertain in your guest face they have littl recours',
       'mention_us and a realli big bad thing about it',
       'mention_us serious would pay a flight for seat that have thi play realli the onli bad thing about fli VA',
       'mention_us ye nearli everi time I fli VX thi go away pos_emo',
       'mention_us realli miss a prime opportun for men without hat parodi there url',
       'mention_us well I now I DO pos_emo',
       'mention_us it wa amaz and arriv an hour earli too good to me',
       'mention_us did you know that suicid is the second lead caus of death among teen',
       'mention_us I pretti graphic so much better than minim iconographi pos_emo',
       'mention_us thi is such a great deal alreadi think about my trip to mention_us I even gone on my trip ye

In [ ]:
x =' '.join(all_text)
x

'mention_us what mention_us said mention_us plu ad commerci to the experi tacki mention_us I today must mean I need to take anoth trip mention_us realli aggress to blast obnoxi entertain in your guest face they have littl recours mention_us and a realli big bad thing about it mention_us serious would pay a flight for seat that have thi play realli the onli bad thing about fli VA mention_us ye nearli everi time I fli VX thi go away pos_emo mention_us realli miss a prime opportun for men without hat parodi there url mention_us well I now I DO pos_emo mention_us it wa amaz and arriv an hour earli too good to me mention_us did you know that suicid is the second lead caus of death among teen mention_us I pretti graphic so much better than minim iconographi pos_emo mention_us thi is such a great deal alreadi think about my trip to mention_us I even gone on my trip yet p mention_us mention_us fli your fabul seduct sky again U take all the stress away from travel url mention_us thank mention_u

In [ ]:
x = x.split()
x[0:5]

['mention_us', 'what', 'mention_us', 'said', 'mention_us']

In [ ]:
from collections import Counter

In [ ]:
Counter(label)

Counter({'negative': 9178, 'neutral': 3099, 'positive': 2363})

In [ ]:
Count = Counter(x)
vocab = sorted(Count, key=Count.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

In [ ]:
encoded = []
for i in range(len(all_text)):
  temp = all_text[i].split()
  enc = []
  for j in range(len(temp)):
    enc.append(vocab_to_int[temp[j]])
  encoded.append(enc)

In [ ]:
lenght_en = len(encoded)
lenght_en

14640

In [ ]:
encoded[0:5]

[[1, 60, 1, 214],
 [1, 505, 587, 1104, 2, 3, 184, 4441],
 [1, 4, 96, 710, 340, 4, 58, 2, 107, 145, 177],
 [1, 130, 2782, 2, 1974, 3354, 752, 13, 18, 1700, 973, 56, 17, 457, 2157],
 [1, 9, 6, 130, 448, 208, 278, 80, 16]]

In [ ]:
length = []
for w in encoded:
  length.append(len(w))

In [ ]:
maks_len = max(length) #maximum word after stem process
maks_len

31

In [ ]:
pad = np.zeros((lenght_en,maks_len),dtype=int)
for i in range(lenght_en):
  residu = maks_len - length[i]
  pad[i,residu:] = encoded[i]

In [ ]:
pad[0:4]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    1,   60,    1,  214],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    1,  505,  587, 1104,    2,    3,  184, 4441],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    1,    4,
          96,  710,  340,    4,   58,    2,  107,  145,  177],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    1,  130, 2782,    2, 1974, 3354,
         752,   13,   18, 1700,  973,   56,   17,  457, 2157]])

In [ ]:
encoded_label = []
for labels in label:
  if labels == 'neutral' or labels == 'positive':
    encoded_label.append(1)
  else:
    encoded_label.append(0) 

In [ ]:
print(label[0:5])
print(encoded_label[0:5])

0     neutral
1    positive
2     neutral
3    negative
4    negative
Name: airline_sentiment, dtype: object
[1, 1, 1, 0, 0]


In [ ]:
split = 0.8
split_num = int(len(pad)*split)

train_x,res_x = pad[:split_num],pad[split_num:]
train_y,res_y = encoded_label[:split_num],encoded_label[split_num:]

split_vt = 0.5
split_num_vt = int((len(pad)-split_num)*0.5)
val_x,test_x = pad[:split_num_vt],pad[split_num_vt:]
val_y,test_y = encoded_label[:split_num_vt],encoded_label[split_num_vt:]

In [ ]:
train_x,train_y = np.array(train_x),np.array(train_y)
val_x,val_y = np.array(val_x), np.array(val_y)
test_x,test_y = np.array(test_x), np.array(test_y)

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure the SHUFFLE the training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size,drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size,drop_last=True)

In [ ]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 31])
Sample input: 
 tensor([[   0,    0,    0,  ...,   37,  362, 5664],
        [   0,    0,    0,  ...,   53,   21,  527],
        [   0,    0,    0,  ...,  300, 1731,  126],
        ...,
        [   0,    0,    0,  ...,  423,  207,  574],
        [   0,    0,    0,  ...,   93,   18, 2958],
        [   0,    0,    0,  ...,    6,  452,  278]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
        1, 1])


In [ ]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [ ]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [ ]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 200
hidden_dim = 128
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(8988, 200)
  (lstm): LSTM(200, 128, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [ ]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
# training params

epochs = 20 

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/20... Step: 100... Loss: 0.580318... Val Loss: 0.444598
Epoch: 1/20... Step: 200... Loss: 0.316676... Val Loss: 0.379330
Epoch: 2/20... Step: 300... Loss: 0.354202... Val Loss: 0.335003
Epoch: 2/20... Step: 400... Loss: 0.355953... Val Loss: 0.286646
Epoch: 3/20... Step: 500... Loss: 0.243223... Val Loss: 0.251144
Epoch: 3/20... Step: 600... Loss: 0.337680... Val Loss: 0.218792
Epoch: 3/20... Step: 700... Loss: 0.266051... Val Loss: 0.184347
Epoch: 4/20... Step: 800... Loss: 0.143748... Val Loss: 0.148330
Epoch: 4/20... Step: 900... Loss: 0.255487... Val Loss: 0.131375
Epoch: 5/20... Step: 1000... Loss: 0.069327... Val Loss: 0.106791
Epoch: 5/20... Step: 1100... Loss: 0.239064... Val Loss: 0.083158
Epoch: 6/20... Step: 1200... Loss: 0.074792... Val Loss: 0.070349
Epoch: 6/20... Step: 1300... Loss: 0.092958... Val Loss: 0.061622
Epoch: 6/20... Step: 1400... Loss: 0.082674... Val Loss: 0.065509
Epoch: 7/20... Step: 1500... Loss: 0.165687... Val Loss: 0.027995
Epoch: 7/20... Step

In [ ]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.221
Test accuracy: 0.957
